# Text Summarization

## Introduction
Text Summarization refers to the technique of  shortening long pieces of text. The intention is to create a coherent and fluent summary having only the main points outlined in the document.

We cannot possibly create summaries of all of the text manually, there is a great need for automatic methods

Automatic text summarization is a common problem in machine learning and natural language processing (NLP).

## Importing Necessary Modules 

In [73]:
import math
import numpy
import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
from numpy.linalg import svd as singular_value_decomposition
from nltk.corpus import stopwords
from warnings import warn
from operator import attrgetter
from collections import namedtuple
from nltk.corpus import stopwords
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

True

## Importing Source File

In [74]:
source_file = "original_text.txt"

with open(source_file, "r", encoding='utf-8') as file:
    text = file.readlines()

## Item Count

In [75]:
class ItemsCount(object):
    def __init__(self, value):
        self._value = value

    def __call__(self, sequence):
        if isinstance(self._value, (bytes, str,)):
            if self._value.endswith("%"):
                total_count = len(sequence)
                percentage = int(self._value[:-1])
                # at least one sentence should be chosen
                count = max(1, total_count*percentage // 100)
                return sequence[:count]
            else:
                return sequence[:int(self._value)]
        elif isinstance(self._value, (int, float)):
            return sequence[:int(self._value)]
        else:
            ValueError("Unsuported value of items count '%s'." % self._value)

    def __repr__(self):
        return to_string("<ItemsCount: %r>" % self._value)

## Base Function

In [76]:
SentenceInfo = namedtuple("SentenceInfo", ("sentence", "order", "rating",))

In [77]:
class BaseSummarizer(object):
    
    def __call__(self, document, sentences_count):
        raise NotImplementedError("This method should be overriden in subclass")

    @staticmethod
    def normalize_word(word):
        return word.lower()

    @staticmethod
    def _get_best_sentences(sentences, count, rating, *args, **kwargs):
        rate = rating
        if isinstance(rating, dict):
            assert not args and not kwargs
            rate = lambda s: rating[s]

        infos = (SentenceInfo(s, o, rate(s, *args, **kwargs))
            for o, s in enumerate(sentences))

        # sort sentences by rating in descending order
        infos = sorted(infos, key=attrgetter("rating"), reverse=True)
        # get `count` first best rated sentences
        if not isinstance(count, ItemsCount):
            count = ItemsCount(count)
        infos = count(infos)
        # sort sentences by their order in document
        infos = sorted(infos, key=attrgetter("order"))

        return tuple(i.sentence for i in infos)

## LSA
Latent semantic analysis method extracts hidden semantic structure of words and sentences. LSA uses the context of the input document and extracts information such as which words are used together and which common words are seen in different sentences.

###  Process
1. Sentence Tokenization
2. Word Tokenization
3. Removing Stop Word
4. Input matrix creation
5. Compute Term Frequency
6. Rank The Sentaces


In [78]:
class LsaSummarizer(BaseSummarizer):
    MIN_DIMENSIONS = 3
    REDUCTION_RATIO = 1/1

    _stop_words = list(stopwords.words('english'))

    @property
    def stop_words(self):
        return self._stop_words

    @stop_words.setter
    def stop_words(self, words):
        self._stop_words = words

    def __call__(self, document, sentences_count):

        dictionary = self._create_dictionary(document)
        
        if not dictionary:
            return ()

    ##Tokenize paragraph into individual senteneces
        sentences = sent_tokenize(document)

        matrix = self._create_matrix(document, dictionary)
        matrix = self._compute_term_frequency(matrix)
        u, sigma, v = singular_value_decomposition(matrix, full_matrices=False)

        ranks = iter(self._compute_ranks(sigma, v))
        return self._get_best_sentences(sentences, sentences_count,
            lambda s: next(ranks))

    def _create_dictionary(self, document):
        """Creates mapping key = word, value = row index"""

    ##Tokenize sentences into individual words
        words = word_tokenize(document)
        words = tuple(words)

        words = map(self.normalize_word, words)

        unique_words = frozenset(w for w in words if w not in self._stop_words)

        return dict((w, i) for i, w in enumerate(unique_words))

    def _create_matrix(self, document, dictionary):
        """
        Creates matrix of shape where cells
        contains number of occurences of words (rows) in senteces (cols).
        """
        sentences = sent_tokenize(document)
        words_count = len(dictionary)
        sentences_count = len(sentences)
        if words_count < sentences_count:
            message = (
                "Number of words (%d) is lower than number of sentences (%d). "
                "LSA algorithm may not work properly."
            )
            warn(message % (words_count, sentences_count))

        matrix = numpy.zeros((words_count, sentences_count))
        for col, sentence in enumerate(sentences):
            words = word_tokenize(sentence)
            for word in words:
                # only valid words is counted (not stop-words, ...)
                if word in dictionary:
                    row = dictionary[word]
                    matrix[row, col] += 1

        return matrix

    def _compute_term_frequency(self, matrix, smooth=0.4):
        """
        Computes TF metrics for each sentence (column) in the given matrix and  normalize 
        the tf weights of all terms occurring in a document by the maximum tf in that document 
        """
        assert 0.0 <= smooth < 1.0

        max_word_frequencies = numpy.max(matrix, axis=0)
        rows, cols = matrix.shape
        for row in range(rows):
            for col in range(cols):
                max_word_frequency = max_word_frequencies[col]
                if max_word_frequency != 0:
                    frequency = matrix[row, col]/max_word_frequency
                    matrix[row, col] = smooth + (1.0 - smooth)*frequency

        return matrix

    def _compute_ranks(self, sigma, v_matrix):
        assert len(sigma) == v_matrix.shape[0]

        dimensions = max(LsaSummarizer.MIN_DIMENSIONS,
            int(len(sigma)*LsaSummarizer.REDUCTION_RATIO))
        powered_sigma = tuple(s**2 if i < dimensions else 0.0
            for i, s in enumerate(sigma))

        ranks = []
        
        for column_vector in v_matrix.T:
            rank = sum(s*v**2 for s, v in zip(powered_sigma, column_vector))
            ranks.append(math.sqrt(rank))

        return ranks

## Function Call

In [79]:
summarizer = LsaSummarizer()

stopwords = stopwords.words('english')
summarizer.stop_words = stopwords
summary =summarizer(text[0], 3)

print("====== Original text =====")
print(text)
print("====== End of original text =====")



print("\n========= Summary =========")

print(" ".join(summary))
print("========= End of summary =========")

====== Original text =====
['As the days went on, and the special was released, and I sat through Dave’s storytelling and ranting on a fixation of now three different specials (trans women, and his feelings about them and their existence), I grew more and more annoyed. I’ve been a fan of Dave Chappelle since I saw Killin’ Them Softly, specifically the joke about Oscar the Grouch and how shitty everyone on Sesame Street treats him. I haven’t revisited that special in years, and I can still recite that entire act out in time with the cadence. I know it. It’s a profound and deeply hilarious moment that proved his genius in the pre-9/11 afterglow of the ’90s and I was always excited to discover his roles in movies before and after that.\n', 'Anyway, I planned to scalp the tickets and not go, but I made that decision on Thursday thinking the show was Friday night. It wasn’t. With just three-and-a-half hours to show time, I decided to go. I knew it wasn’t going to be a stand-up set, or the N